<a href="https://colab.research.google.com/github/jota1919/MADS_Grupo6/blob/main/Flask_aluguer_imoveis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyngrok flask
!pip install folium
!pip install gspread
!ngrok authtoken 2xaXwGP2RC9L0F4Ejti15YRP4Io_7msuBBpvFpd7o71Hpyk7t

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
from flask import Flask, request, render_template_string, redirect, url_for
from pyngrok import ngrok
import gspread
import pandas as pd
from google.auth import default
from google.colab import auth
import folium

# Autenticação Google
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Google Sheets
spreadsheet_id = '1c0iF-CUNAgAHKCxIOGEPIJqtYTE96I6KZGAmzEIVzr8'
sh = gc.open_by_key(spreadsheet_id)

df_imoveis = pd.DataFrame(sh.worksheet("Imoveis").get_all_records())
df_clientes = pd.DataFrame(sh.worksheet("Clientes").get_all_records())
df_reservas = pd.DataFrame(sh.worksheet("Reservas").get_all_records())

# Flask
app = Flask(__name__)
port = 5000
public_url = ngrok.connect(port)
print(f"Aplicação a rodar em: {public_url}")

SENHA_PRIVADA = "1234"

# Função para gerar o mapa
def gerar_mapa():
    mapa = folium.Map(location=[38.7169, -9.1399], zoom_start=6)
    for _, row in df_imoveis.iterrows():
        try:
            lat = float(row["Latitude"])
            lon = float(row["Longitude"])
            popup = f"{row['Descrição']}<br>{row['Preço/Noite (€)']}€/noite"
            folium.Marker([lat, lon], popup=popup).add_to(mapa)
        except:
            continue
    return mapa._repr_html_()

TEMPLATE_BASE = """
<!DOCTYPE html>
<html lang="pt">
<head>
    <meta charset="UTF-8">
    <title>{{ titulo }}</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css">
    <style>
        .table th, .table td {
            text-align: center;
            vertical-align: middle;
        }
        body {
            margin: 20px;
        }
        .nav-buttons {
            margin-top: 20px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>{{ titulo }}</h1>
        <div class="nav-buttons">
            {% if titulo != "Área Privada - Login" %}
                <a class="btn btn-primary" href="/">Início</a>
            {% endif %}
            {% if titulo == "Imóveis Disponíveis" %}
                <a class="btn btn-secondary" href="/login">Área Privada</a>
            {% endif %}
        </div>
        <hr>
        {{ conteudo|safe }}
    </div>
</body>
</html>
"""



@app.route("/")
def home():
    mapa_html = gerar_mapa()
    tabela_html = df_imoveis[["Localização", "Preço/Noite (€)", "Descrição"]].to_html(classes='table table-bordered table-hover', index=False, border=0)
    conteudo = f"""
    <div id="map">{mapa_html}</div>
    <div class="tabela-wrapper">
        <h2>Lista de Imóveis</h2>
        {tabela_html}
    </div>
    """
    return render_template_string(TEMPLATE_BASE, titulo="Imóveis Disponíveis", conteudo=conteudo)

@app.route("/login", methods=["GET", "POST"])
def login():
    if request.method == "POST":
        senha = request.form.get("senha")
        if senha == SENHA_PRIVADA:
            return redirect(url_for("privado"))
        else:
            return render_template_string(TEMPLATE_BASE, titulo="Login", conteudo="<p>Senha incorreta!</p>")
    conteudo = """
    <form method="POST" class="mt-4">
        <input type="password" name="senha" class="form-control mb-2" placeholder="Digite a senha">
        <button type="submit" class="btn btn-primary">Entrar</button>
    </form>
    """
    return render_template_string(TEMPLATE_BASE, titulo="Login - Área Privada", conteudo=conteudo)

@app.route("/privado")
def privado():
    tabela_clientes = df_clientes.to_html(classes='table table-striped', index=False)
    tabela_reservas = df_reservas.to_html(classes='table table-striped', index=False)
    tabela_imoveis = df_imoveis.to_html(classes='table table-striped', index=False)
    conteudo = f"""
    <h1>Área Privada</h1>
    <h2>Clientes</h2>
    {tabela_clientes}
    <h2>Reservas</h2>
    {tabela_reservas}
    <h2>Imóveis - Detalhes</h2>
    {tabela_imoveis}
    """
    return render_template_string(TEMPLATE_BASE, titulo="Área Privada", conteudo=conteudo)

app.run(port=port)

Aplicação a rodar em: NgrokTunnel: "https://3acc-35-243-144-195.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [28/May/2025 09:38:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/May/2025 09:38:43] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [28/May/2025 09:40:02] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/May/2025 09:40:09] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [28/May/2025 09:40:10] "GET /privado HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/May/2025 09:40:45] "GET / HTTP/1.1" 200 -
